In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 999)

In [2]:
df = pd.read_csv('Fire_Department_Calls_for_Service.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,20,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(4715699, 34)

In [12]:
#memory usage: 1.2+ GB
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4715699 entries, 0 to 4715698
Data columns (total 34 columns):
Call Number                             int64
Unit ID                                 object
Incident Number                         int64
Call Type                               object
Call Date                               object
Watch Date                              object
Received DtTm                           object
Entry DtTm                              object
Dispatch DtTm                           object
Response DtTm                           object
On Scene DtTm                           object
Transport DtTm                          object
Hospital DtTm                           object
Call Final Disposition                  object
Available DtTm                          object
Address                                 object
City                                    object
Zipcode of Incident                     float64
Battalion                               ob

In [7]:
#makes memory usage: 7.9 GB due to more detailed numbers to get an accurate number.
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4715699 entries, 0 to 4715698
Data columns (total 34 columns):
Call Number                             int64
Unit ID                                 object
Incident Number                         int64
Call Type                               object
Call Date                               object
Watch Date                              object
Received DtTm                           object
Entry DtTm                              object
Dispatch DtTm                           object
Response DtTm                           object
On Scene DtTm                           object
Transport DtTm                          object
Hospital DtTm                           object
Call Final Disposition                  object
Available DtTm                          object
Address                                 object
City                                    object
Zipcode of Incident                     float64
Battalion                               ob

In [13]:
for dtype in ['float','int','object','bool']:
    selected_dtype = df.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 23.99 MB
Average memory usage for int columns: 28.78 MB
Average memory usage for object columns: 282.68 MB
Average memory usage for bool columns: 2.25 MB


We can use the function pd.to_numeric() to downcast our numeric types. We'll use DataFrame.select_dtypes to select only the integer columns, then we'll optimize the types and compare the memory usage.

In [14]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

df_int = df.select_dtypes(include=['int'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(df_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([df_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

143.91 MB
44.97 MB


,before,after
uint8,NaN,2.0
uint32,NaN,2.0
int64,4.0,NaN


Lets do the same thing with our float columns.

In [18]:
df_float = df.select_dtypes(include=['float'])
converted_float = df_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(df_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([df_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

71.96 MB
35.98 MB


,before,after
float32,NaN,2.0
float64,2.0,NaN


In [19]:
optimized_df = df.copy()

optimized_df[converted_int.columns] = converted_int
optimized_df[converted_float.columns] = converted_float

print(mem_usage(df))
print(mem_usage(optimized_df))

1191.77 MB
1056.85 MB


write a loop to iterate over each object column, check if the number of unique values is less than 50%, and if so, convert it to the category type.

In [22]:
converted_obj = pd.DataFrame()
df_obj = df.select_dtypes(include=['object']).copy()

for col in df_obj.columns:
    num_unique_values = len(df_obj[col].unique())
    num_total_values = len(df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]

In [23]:
print(mem_usage(df_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([df_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

971.40 MB
743.80 MB


,before,after
object,27.0,4
category,NaN,2
category,NaN,2
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1


In [5]:
optimized_df.head()

,0,1,2,3,4
Call Number,1030101,1030104,1030106,1030107,1030108
Unit ID,E18,M14,M36,E01,RS1
Incident Number,306091,30612,30614,30615,30616
Call Type,Medical Incident,Medical Incident,Medical Incident,Alarms,Medical Incident
Call Date,04/12/2000,04/12/2000,04/12/2000,04/12/2000,04/12/2000
Watch Date,04/12/2000,04/12/2000,04/12/2000,04/12/2000,04/12/2000
Received DtTm,04/12/2000 09:00:29 PM,04/12/2000 09:09:02 PM,04/12/2000 09:09:44 PM,04/12/2000 09:13:47 PM,04/12/2000 09:14:43 PM
Entry DtTm,04/12/2000 09:01:40 PM,04/12/2000 09:10:17 PM,04/12/2000 09:10:56 PM,04/12/2000 09:13:51 PM,04/12/2000 09:16:11 PM
Dispatch DtTm,04/12/2000 09:02:00 PM,04/12/2000 09:10:29 PM,04/12/2000 09:11:47 PM,04/12/2000 09:14:13 PM,04/12/2000 09:16:24 PM
Response DtTm,NaN,04/12/2000 09:12:11 PM,NaN,04/12/2000 09:15:58 PM,04/12/2000 09:18:20 PM


In [25]:
optimized_df.head().T

,0,1,2,3,4
Call Number,1030101,1030104,1030106,1030107,1030108
Unit ID,E18,M14,M36,E01,RS1
Incident Number,306091,30612,30614,30615,30616
Call Type,Medical Incident,Medical Incident,Medical Incident,Alarms,Medical Incident
Call Date,04/12/2000,04/12/2000,04/12/2000,04/12/2000,04/12/2000
Watch Date,04/12/2000,04/12/2000,04/12/2000,04/12/2000,04/12/2000
Received DtTm,04/12/2000 09:00:29 PM,04/12/2000 09:09:02 PM,04/12/2000 09:09:44 PM,04/12/2000 09:13:47 PM,04/12/2000 09:14:43 PM
Entry DtTm,04/12/2000 09:01:40 PM,04/12/2000 09:10:17 PM,04/12/2000 09:10:56 PM,04/12/2000 09:13:51 PM,04/12/2000 09:16:11 PM
Dispatch DtTm,04/12/2000 09:02:00 PM,04/12/2000 09:10:29 PM,04/12/2000 09:11:47 PM,04/12/2000 09:14:13 PM,04/12/2000 09:16:24 PM
Response DtTm,NaN,04/12/2000 09:12:11 PM,NaN,04/12/2000 09:15:58 PM,04/12/2000 09:18:20 PM


In [26]:
optimized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4715699 entries, 0 to 4715698
Data columns (total 34 columns):
Call Number                             uint32
Unit ID                                 object
Incident Number                         uint32
Call Type                               object
Call Date                               object
Watch Date                              object
Received DtTm                           object
Entry DtTm                              object
Dispatch DtTm                           object
Response DtTm                           object
On Scene DtTm                           object
Transport DtTm                          object
Hospital DtTm                           object
Call Final Disposition                  object
Available DtTm                          object
Address                                 object
City                                    object
Zipcode of Incident                     float32
Battalion                               

ideas: https://www.dataquest.io/blog/pandas-big-data/